In [627]:
import pandas as pd; import numpy as np; import matplotlib.pyplot as plt
import panel as pn
pn.extension(comms='vscode')
%matplotlib inline

In [628]:
total_rounds = 2
max_cap = 5
max_horizon = 40

stationary_arrival_rate = 0.3

peak = 0.61538
non_stationary_arrival_rates = np.concatenate((np.linspace(0,peak,10), np.linspace(peak,0,31)[1:]))
non_stationary_arrival_rates.mean()

0.29999775

In [629]:
class DataHandler():

    def __init__(self,max_cap,max_horizon,total_rounds):
        self.total_revenue = 0
        self.current_wtp = 0

        self.current_occ = 0 
        self.current_time = 0

        self.max_horizon = max_horizon
        self.max_cap = max_cap

        self.wtps = np.random.randint(50, 101, size=data.max_horizon)

        self.current_round = 1
        self.total_rounds = total_rounds

    def reset(self):
        self.total_revenue = 0
        self.current_wtp = 0

        self.current_occ = 0 
        self.current_time = 0

        self.wtps = np.random.randint(50, 101, size=data.max_horizon)

        self.current_round += 1


    @property
    def leftover_capacity(self):
        return self.max_cap - self.current_occ

    @property
    def leftover_time(self):
        return self.max_horizon - self.current_time

data = DataHandler(max_cap,max_horizon,total_rounds)

In [630]:
header = pn.pane.Markdown("# Panel Dashboarding")
task_descr = pn.pane.Markdown("""
###Accept or Reject arriving Customers<br>
You start managing customers when there are {} days left until departure.<br>
Each day, at most one customer requests a seat with given probability.<br>
The customer indicates a maximum price he is willing to pay.""".format(max_horizon))

In [631]:
app = pn.Column(header, task_descr)

In [632]:
round = pn.pane.Markdown("## Round [{} / {}]".format(data.current_round,data.total_rounds))
app.append(round)

In [633]:
total_revenue = pn.pane.Markdown("##Total Revenue: {} €".format(data.total_revenue))
app.append(total_revenue)

In [634]:
# leftover capacity bar
leftover_capacity = pn.pane.Markdown(f"### Leftover Capacity: {data.leftover_capacity}")
leftover_capacity_bar = pn.indicators.Progress(name='Progress', value=0,max=data.max_cap, width=200)
leftover_element = pn.Column(leftover_capacity, leftover_capacity_bar)


# days until departure bar
days_until_departure = pn.pane.Markdown(f"### Days Until Departure: {data.leftover_time}")
days_until_departure_bar = pn.indicators.Progress(name='Progress', value=0,max=data.max_horizon, width=200)
days_until_departure_element = pn.Column(days_until_departure, days_until_departure_bar)

bars = pn.Row(leftover_element, days_until_departure_element)

progress = pn.indicators.Progress(name='Progress', value=20, width=200)

app.append(bars)


In [635]:
customer_information = pn.pane.Markdown("## A new customer arrives and is willing to pay {}".format(data.wtps[0]))
app.append(customer_information)

In [636]:
accept_button = pn.widgets.Button(name='Accept', button_type='success')
reject_button = pn.widgets.Button(name='Reject', button_type='danger')
ok_button = pn.widgets.Button(name='OK', button_type='primary')
next_round_button = pn.widgets.Button(name='Next Round', button_type='primary')
button_row = pn.Row(accept_button, reject_button)


In [637]:

app.append(button_row)

# when clicking accept button, update the progress bar
def accept(event):

    data.current_occ +=1
    data.current_time +=1
    
    leftover_capacity_bar.value = data.current_occ
    days_until_departure_bar.value = data.current_time

    leftover_capacity.value = f"### Leftover Capacity: {data.leftover_capacity}"
    days_until_departure.object = f"### Days Until Departure: {data.leftover_time}"

    data.total_revenue += data.current_wtp
    total_revenue.object = "##Total Revenue: {} €".format(data.total_revenue)

    data.current_wtp = 0
    new_day()

def reject(event):
    data.current_time +=1
    days_until_departure_bar.value = data.current_time
    days_until_departure.object = f"### Days Until Departure: {data.leftover_time}"
    new_day()
    
def new_day(event=None):

    if data.leftover_capacity == 0 or data.leftover_time == 0:

        if data.current_round < data.total_rounds:
            button_row.clear()
            button_row.append(next_round_button)
            customer_information.object = "###The flight is fully booked or departing today."

        else:
            button_row.clear()
            customer_information.object = """
            ###The flight is fully booked or departing today.
            Your data has been saved. Thank you for participating. You can close this window now."""


    else:
        customer_arrives = np.random.choice([True, False], p=[stationary_arrival_rate, 1-stationary_arrival_rate])
        if customer_arrives:
            data.current_wtp = data.wtps[data.current_time]
            customer_information.object = "## A new customer arrives and is willing to pay {}".format(data.current_wtp)
            button_row.clear()
            button_row.append(accept_button)
            button_row.append(reject_button)

        else:
            customer_information.object = "## No customer arrives today"
            button_row.clear()
            button_row.append(ok_button)

def reset(event=None):

    data.reset()

    round.object = "## Round [{} / {}]".format(data.current_round,data.total_rounds)

    total_revenue.object = "##Total Revenue: {} €".format(data.total_revenue)

    leftover_capacity_bar.value = data.current_occ
    days_until_departure_bar.value = data.current_time

    leftover_capacity.value = f"### Leftover Capacity: {data.leftover_capacity}"
    days_until_departure.object = f"### Days Until Departure: {data.leftover_time}"

    button_row.clear()
    button_row.append(accept_button)
    button_row.append(reject_button)



next_round_button.on_click(reset)
accept_button.on_click(accept)
reject_button.on_click(reject)
ok_button.on_click(reject)


Watcher(inst=Button(button_type='primary', name='OK'), cls=<class 'panel.widgets.button.Button'>, fn=<function reject at 0x0000020F0B081AF0>, mode='args', onlychanged=False, parameter_names=('clicks',), what='value', queued=False, precedence=0)

In [638]:
app

BokehModel(combine_events=True, render_bundle={'docs_json': {'c3984d91-0222-481a-b49f-ecdc65fdde5c': {'defs': …

In [113]:
# from bokeh.plotting import figure

# p1 = figure(width=300, height=300)
# p1.line([1, 2, 3], [1, 2, 3])

# tabs = pn.Tabs(p1)

# # Add a tab
# tabs.append(('Slider', pn.widgets.FloatSlider()))

# # Add multiple tabs
# tabs.extend([
#     ('Text', pn.widgets.TextInput()),
#     ('Color', pn.widgets.ColorPicker())
# ])

# tabs